In [3]:
%pip install pykeen
%pip install torch
%pip install numpy==1.26.4
# %pip install cuda
%pip install matplotlib
# %pip install torch==2.3.0+cu121 torchvision torchaudio
# %pip install --upgrade pip
%pip install pykeen
%pip install rdflib
%pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 817.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 7.3 MB/s eta 0:00:00


In [1]:
import pandas as pd

cluster_groups = pd.read_csv('cluster_groups.csv')
cluster_groups.to_dict().values()

dict_values([{0: '[10000032, 10001217, 10001725, 10002428, 10002495, 10002930, 10003046, 10003400, 10004235, 10004422, 10004457, 10004720, 10004733, 10005348, 10005817, 10005866, 10005909, 10006053, 10006580, 10007058, 10007795, 10007818, 10007928, 10008287, 10008454, 10009035, 10009049, 10009628, 10010471, 10010867, 10011398, 10012552, 10012853, 10013049]', 1: '[10029291, 10029484, 10031404, 10031757, 10032725, 10035185, 10035631, 10036156, 10037861, 10037928, 10037975, 10038081, 10038933, 10038992, 10038999, 10039708, 10039831, 10039997, 10040025]', 2: '[10014078, 10014354, 10014729, 10015272, 10015860, 10015931, 10016150, 10016742, 10016810, 10017492, 10018081, 10018328, 10018423, 10018501, 10018845, 10019003, 10019172, 10019385, 10019568, 10019777, 10019917, 10020187, 10020306, 10020640, 10020740, 10020786, 10020944, 10021118, 10021312, 10021487, 10021666, 10021938, 10022017, 10022041, 10022281, 10022880, 10023117, 10023239, 10023771, 10024043, 10025463, 10025612, 10026255, 1002635

In [2]:
#### Parsing the Knowledge graph

import rdflib
import numpy as np
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
import os

rdf_graph = rdflib.Graph()

KG_Path = "/content/100_patients.ttl"
rdf_graph.parse(KG_Path, format="ttl")

print(f"✅ Loaded {len(rdf_graph):,} triples")

INFO:pykeen.utils:Using opt_einsum


✅ Loaded 7,192,283 triples


In [3]:
#### Creating the triples factory

tf = TriplesFactory.from_labeled_triples(np.array([(str(s), str(p), str(o)) for s, p, o in rdf_graph], dtype=object))

In [4]:
tf.mapped_triples.shape

torch.Size([7192283, 3])

In [5]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
def get_entity_id(model, entity_name, triples_factory):
    entity_id = triples_factory.entity_to_id[entity_name]
    # # Get the embedding tensor for the entity
    # embedding = model.entity_representations[0](indices=torch.tensor([entity_id]))
    embedding = model["entity_representations.0._embeddings.weight"][entity_id]
    # # If you want the embedding as a numpy array:
    embedding_np = embedding.detach()
    return embedding_np

def get_relation_id(model, relation_name, triples_factory):
    relation_id = triples_factory.relation_to_id[relation_name]
    # # Get the embedding tensor for the relation
    # embedding = model.relation_representations[0](indices=torch.tensor([relation_id]))
    embedding = model["relation_representations.0._embeddings.weight"][relation_id]
    # # If you want the embedding as a numpy array:
    embedding_np = embedding.detach()
    return embedding_np

In [ ]:
from enum import Enum
from tqdm import tqdm

SPHN = "https://biomedit.ch/rdf/sphn-schema/sphn#"
FOAF = "http://xmlns.com/foaf/0.1/"

class ContextLevel(Enum):
    LEVEL_1 = 1
    LEVEL_2_ALL_AVG = 21
    LEVEL_2_WEIGHTED_AVG = 22

def get_lvl2_context(patient_id, model, tf, context_lvl:ContextLevel):
    # administrative_case_results, labevent_results
    administrative_case_results = rdf_graph.query(
        "SELECT DISTINCT ?administrative_case WHERE {"
        "?administrative_case <"+SPHN+"hasSubjectPseudoIdentifier> <http://example.org/subject/"+str(patient_id)+"> ."
        "?a <"+SPHN+"hasAdministrativeCase> ?administrative_case ."
        "}")
    unique_administrative_cases = pd.DataFrame(list(administrative_case_results)).iloc[:,0]

    labevent_results = rdf_graph.query(
        "SELECT DISTINCT ?labevent WHERE {"
        # "?subject ?predicate <http://example.org/subject/"+str(patient_id)+"> ."
        "?sample <"+SPHN+"hasSubjectPseudoIdentifier> <http://example.org/subject/"+str(patient_id)+"> ."
        "?labevent <"+SPHN+"hasSample> ?sample ."
        "}")
    unique_labevents = pd.DataFrame(list(labevent_results)).iloc[:,0]
    if context_lvl == ContextLevel.LEVEL_2_WEIGHTED_AVG:
        administrative_case_embeddings = [get_entity_id(model, str(entity), tf) for entity in unique_administrative_cases]
        labevent_embeddings = [get_entity_id(model, str(entity), tf) for entity in unique_labevents]
        administrative_case_embeddings = torch.mean(torch.stack(administrative_case_embeddings), dim=0)
        labevent_embeddings = torch.mean(torch.stack(labevent_embeddings), dim=0)
        patient_context = torch.mean(torch.stack([administrative_case_embeddings, labevent_embeddings]), dim=0)
        return patient_context
    else:
        administrative_case_embeddings = [get_entity_id(model, str(entity), tf) for entity in unique_administrative_cases]
        labevent_embeddings = [get_entity_id(model, str(entity), tf) for entity in unique_labevents]
        stacked = torch.stack(administrative_case_embeddings + labevent_embeddings)
        patient_context = torch.mean(stacked, dim=0)
        return patient_context

def get_lvl1_context(patient_id, model, tf):
    patient_url = "http://example.org/subject/" + str(patient_id)
    patient_context = get_entity_id(model, patient_url,tf)
    transformed_patient_context = patient_context
    return transformed_patient_context

def get_patient_context(model, patient_id, context_lvl:ContextLevel = ContextLevel.LEVEL_1):
    if context_lvl == ContextLevel.LEVEL_1:
        return get_lvl1_context(patient_id, model, tf)
    elif context_lvl in (ContextLevel.LEVEL_2_ALL_AVG, ContextLevel.LEVEL_2_WEIGHTED_AVG):
        return get_lvl2_context(patient_id, model, tf, context_lvl)

In [10]:
def fetch_icd_codes(subject_id):
  icd_codes = []
  query = """
  PREFIX sphn: <https://biomedit.ch/rdf/sphn-schema/sphn#>


  SELECT ?icd_code ?subject_id ?diagnosis
  WHERE {
      ?diagnosis sphn:hasSubjectPseudoIdentifier ?subject_id .
      ?diagnosis sphn:hasCode ?icd_code .
      FILTER (?subject_id = '%s')
  }
  """

  qres = rdf_graph.query(query % subject_id)
  for row in qres:
    icd_codes.append(row[0])

  return icd_codes

In [60]:
icd_codes_dict = {}


q = f"""
PREFIX sphn: <https://biomedit.ch/rdf/sphn-schema/sphn#>

SELECT ?icd_code ?diagnosis
WHERE {{
    ?diagnosis sphn:hasSubjectPseudoIdentifier ?subject_id .
    ?diagnosis sphn:hasCode ?icd_code .
}}
"""
entity_id = rdflib.URIRef("http://example.org/subject/10001217")

qres = rdf_graph.query(q,initBindings={ 'subject_id' :entity_uri})
for row in qres:
  print(row[0])

In [11]:
fetch_icd_codes(10000032)

[]

In [ ]:
subject_id_icd_code_mapping = {}

for patient_list in cluster_groups:
  for patient in patient_list:
    icd_codes = fetch_icd_codes(patient)
    subject_id_icd_code_mapping[patient].append(icd_codes)

In [1]:
for patient_list in cluster_groups:
    for patient in patient_list:
        icd_codes = fetch_icd_codes(patient)  # This returns a list
        subject_id_icd_code_mapping[patient].extend(icd_codes)  # Add all codes

NameError: name 'cluster_groups' is not defined